In [3]:
! uv add groq
#!pip install Groq

Resolved 122 packages in 0.55ms
Audited 117 packages in 0.02ms


In [4]:
import groq
from groq import Groq
import os
from getpass import getpass

ModuleNotFoundError: No module named 'groq'

In [ ]:
print(groq.__version__)

In [19]:
from pprint import pprint as pp
# Set a custom width for pretty-printing
def pprint(data, width=80):
    """Pretty print data with a specified width."""
    pp(data, width=width)# List of model identifiers to query


### Setting Up API Keys

In [20]:
# Enter API key
api_key = getpass('Enter your Groq API key: ')

In [24]:
# Configure the default for all requests:
client = Groq(
    max_retries=2,# default is 2
    api_key=api_key,
     # 20 seconds (default is 1 minute)
    timeout=50.0,
)

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "generate a sample codemeta.json file",
        }
    ],
    model="llama3-8b-8192",
)
print(chat_completion.choices[0].message.content)

In [25]:
# Instructions for Groq 
instructions = '''
Please provide a comprehensive list of all internationally recognized sovereign states. 
    Format the response as a clean, alphabetized list of country names. 
    Include:
    - All 193 UN member states
    - Internationally recognized sovereign states
    - Alphabetized order
    
    Do not include:
    - Dependent territories
    - Disputed regions
    - Non-sovereign states
'''


In [ ]:
country_list= ''
try:
    response = client.chat.completions.with_raw_response.create(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": instructions,
            },
        ],
        model="llama3-8b-8192",
    )
    #print(response.headers.get('X-My-Header'))
    for line in response.iter_lines():
        country_list=country_list+line
        print(line)
except groq.APIConnectionError as e:
    print("The server could not be reached")
    print(e.__cause__)  # an underlying Exception, likely raised within httpx.
except groq.RateLimitError as e:
    print("A 429 status code was received; we should back off a bit.")
except groq.APIStatusError as e:
    print("Another non-200-range status code was received")
    print(e.status_code)
    print(e.response)

In [ ]:
import pandas as pd
import json

In [ ]:
# Parse the JSON
data = json.loads(country_list)

# Extract the content containing the countries
content = data['choices'][0]['message']['content']


# Split the content into lines and remove the header
countries = content.split('\n')[2:]  # Skip the first two lines (header and empty line)

# Clean the data by removing the incorrect rows
# Remove empty strings and the note at the end
countries = [country for country in countries if country and not country.startswith('Please')]

# Create a DataFrame
df = pd.DataFrame(countries, columns=['Country'])

# Add an index column starting from 1
df.index = df.index + 1

# Display the DataFrame
display(df.head(15))

# Print some basic statistics
print(f"\nTotal number of countries: {len(df)}")

In [ ]:
#| label: celllabel1

df.tail()

### Generate a CC by 4.0 License File for Github

In [ ]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Generate a CC by 4.0 License File for Github",
        }
    ],
    model="llama3-8b-8192",
)
print(chat_completion.choices[0].message.content)